### Handle imports

In [ ]:
import csv
import json
import os
import sys
import time

import numpy as np
from tqdm.auto import tqdm

### Define Puzzle objects

In [ ]:
class Move:
    
    def __init__(self, name: str, permutation: np.ndarray):
        self.name = name
        self.permutation = permutation
    
    def get_reverse(self):
        assert not self.name.startswith("-"), "move is already a reverse move"
        return Move("-" + self.name, np.argsort(self.permutation))
    
    def get_zero_move(self):
        return Move("0", np.arange(len(self.permutation)))
    
    def __getitem__(self, i):
        return self.permutation[i]
        
    def __call__(self, a: np.ndarray, inplace: bool = True):
        assert isinstance(a, np.ndarray), "argument must be a numpy array"
        if inplace:
            a[:] = a[self.permutation]
        else:
            return a[self.permutation]
        
        
class MoveSet:
    
    class MoveSetIter:
        
        def __init__(self, *moves: Move):
            self.moves = list(moves)
            self.index, self.stop_index = -1, len(moves)
            
        def __next__(self):
            self.index += 1
            if self.index == self.stop_index:
                raise StopIteration
            return self.moves[self.index]
        
        def __iter__(self):
            return self
            
    
    def __init__(self, type_: str, *moves: Move):
        self.type_ = type_
        self.moves = {
            move.name: move
            for move in moves
        }
        
    def __iter__(self):
        return iter(MoveSet.MoveSetIter(*self.moves.values()))
        
    def __getitem__(self, name: str):
        return self.moves[name]
    
    def __len__(self):
        return len(self.moves)


class Puzzle:
    
    def __init__(self, id: int, type_: str, 
                 solution: np.ndarray, initial_state: np.ndarray, 
                 num_wildcards: int):
        self.id = id
        self.type_ = type_
        self.solution = solution
        self.initial_state = initial_state
        self.current_state = initial_state
        self.num_wildcards = num_wildcards
        
        self.solution_path: list[np.ndarray] = [initial_state]
        self.applied_moves: list[np.ndarray] = []
        
    def apply_moves(self, *moves: Move):
        for move in moves:
            self.current_state = move(self.current_state, inplace=False)
            self.solution_path.append(self.current_state)
            self.applied_moves.append(move.permutation.copy())
        
    def print_solution_path(self, include_moves: bool = False, symbol_dict: dict[int, str] | None = None):
        n_moves = len(self.applied_moves)
        for i, state in enumerate(self.solution_path):
            print(f"State {i}:", end="")
            if symbol_dict is not None:
                print([symbol_dict[s] for s in state], end="")
            else:
                print(state, end="")
            print()
            
            if include_moves and i < n_moves:
                print(f"\tMove ({i}->{i+1}):", self.applied_moves[i])
                
    def is_solved(self) -> bool:
        return np.array_equal(self.current_state, self.solution)
                
    def reset(self):
        self.current_state = self.initial_state
        self.solution_path = [self.initial_state]
        self.applied_moves = []
        
    def __len__(self):    
        return len(self.solution)
                

### Load data

In [ ]:
def get_reader(path: str):
    return csv.reader(open(path, mode="r", encoding="utf-8"), delimiter=",")


Map symbols (strings) to ids (integers)

In [ ]:
reader = get_reader("data/puzzles.csv")
next(reader)  # skip header

symid_symbol: dict[int, str] = dict()
symbol_symid: dict[str, int] = dict()
symid: int = 0
for line in reader:
    solution_state = line[2].split(";")
    for symbol in solution_state:
        if symbol not in symbol_symid:
            symid += 1
            symbol_symid[symbol] = symid
            symid_symbol[symid] = symbol


Load puzzles

In [ ]:
reader = get_reader("data/puzzles.csv")
next(reader)  # skip header

id_puzzle: dict[int, Puzzle] = dict()
for line in reader:
    id = int(line[0])
    type_ = line[1]
    solution = np.array([symbol_symid[symbol] for symbol in line[2].split(";")])
    state = np.array([symbol_symid[symbol] for symbol in line[3].split(";")])
    num_wildcards = int(line[4])
    
    id_puzzle[id] = Puzzle(id, type_, solution, state, num_wildcards)


Load moves

In [ ]:
# set csv max field size to handle reading in the puzzle info
csv_limit = sys.maxsize
while True:
    try:
        csv.field_size_limit(csv_limit)
        break
    except OverflowError:
        csv_limit = int(csv_limit / 10)
        
# read puzzle info
reader = get_reader("data/puzzle_info.csv")
next(reader)  # skip header

type_moveset: dict[str, MoveSet] = dict()
for line in reader:
    type_ = line[0]
    moves = json.loads(line[1].replace("'", '"'), )
    move_objects = []
    for name, permutation in moves.items():
        move = Move(name, np.array(permutation))
        reverse_move = move.get_reverse()
        move_objects.append(move)
        move_objects.append(reverse_move)
        
    if type_ in type_moveset:
        raise AssertionError(f"type {type_} already in type_moveset")
    
    type_moveset[type_] = MoveSet(type_, *move_objects)

Load sample submission

In [ ]:
puzzleid_moveids: dict[int, list[str]] = dict()
reader = get_reader("submissions/sample_submission.csv")
next(reader)  # skip header

for line in reader:
    id = int(line[0])
    moveids = line[1].split(".")
    puzzleid_moveids[id] = moveids
    

Verify sample submission

In [ ]:
def score_submission(puzzleid_moveids: dict[int, list[str]]):
    score = 0
    for id, moveids in tqdm(puzzleid_moveids.items()):
        puzzle = id_puzzle[id]
        puzzle.reset()
        
        moveset = type_moveset[puzzle.type_]
        for moveid in moveids:
            try:
                puzzle.apply_moves(moveset[moveid])
            except KeyError:
                raise AssertionError(f"move with id {moveid} is not an allowed move for puzzle {id}")
        num_wrong_facelets = np.sum(puzzle.solution != puzzle.current_state)
        if num_wrong_facelets > puzzle.num_wildcards:
            raise AssertionError(f"submitted moves do not solve puzzle {id}")
        
        puzzle.reset()
        
        score += len(moveids)
        
    return score

In [ ]:
# --- SCORE SAMPLE SUBMISSION ---
# [puzzle.reset() for puzzle in id_puzzle.values()]
# print("Sample submission score:", score_submission(puzzleid_moveids))

In [ ]:
def get_num_faces(puzzle: Puzzle) -> int:
    faces = puzzle.type_.split("_")[1] 
    return sum(int(face) for face in faces.split("/"))


def get_face_size(puzzle: Puzzle) -> int:
    faces = puzzle.type_.split("_")[1]
    return int(faces.split("/")[0])


id = 0  # cube
puzzle = id_puzzle[id]
face_size = get_face_size(puzzle)
n_faces = len(puzzle) // face_size**2

current_cube = puzzle.solution.reshape((-1, face_size**2))
solution_cube = puzzle.solution.reshape((-1, face_size**2))

facelet_channels = np.concatenate(
    tuple(np.atleast_3d((current_cube == facelet_value)).astype(np.float32)
          for facelet_value in np.unique(current_cube)), 
    -1
)

solution_channel = np.atleast_3d(current_cube == solution_cube).astype(np.float32)
wildcard_channel = np.atleast_3d(puzzle.num_wildcards * np.ones_like(solution_channel)).astype(np.float32)

input_channels = np.concatenate((facelet_channels, solution_channel, wildcard_channel), -1)


In [ ]:
def get_cuts(puzzle: Puzzle) -> tuple[int, int]:
    cuts = puzzle.type_.split("_")[1].split("/")
    return int(cuts[0]), int(cuts[1])


id = 380  # globe
puzzle = id_puzzle[id]
lat_cuts, long_cuts = get_cuts(puzzle)
lat_parts, long_parts = lat_cuts + 1, 2 * long_cuts 

current_cube = puzzle.solution.reshape((lat_parts, long_parts))
solution_cube = puzzle.solution.reshape((lat_parts, long_parts))

facelet_channels = np.concatenate(
    tuple(np.atleast_3d((current_cube == facelet_value)).astype(np.float32)
          for facelet_value in np.unique(current_cube)), 
    -1
)

solution_channel = np.atleast_3d(current_cube == solution_cube).astype(np.float32)
wildcard_channel = np.atleast_3d(puzzle.num_wildcards * np.ones_like(solution_channel)).astype(np.float32)

input_channels = np.concatenate((facelet_channels, solution_channel, wildcard_channel), -1)
print(input_channels.shape)

In [ ]:
def get_puzzleid_moveids(path: str) -> dict[int, list[str]]:
    reader = get_reader(path)
    next(reader)  # skip header
    return {int(line[0]): line[-1].split(".")
            for line in reader if "0" in line[3]}


In [ ]:
for file in next(os.walk("solutions"))[2]:
    if "_" in file:
        puzzleid_moveids.update(get_puzzleid_moveids(f"solutions/{file}"))

In [ ]:
print(score_submission(puzzleid_moveids))